In [3]:
def informacoes_auxiliares( 
    informacoes_card_imovel,
    cod_imoveis):
        
    qntd_items = []
    valores = []
    infos_extras_imovel = {}

    for j in range(0, len(informacoes_card_imovel)):
        infos_extras_imovel.setdefault(cod_imoveis[j], [])
    
        for k in range(0, len(informacoes_card_imovel[j])):
            # Salvando as variaveis em listas
            qntd_items.append( informacoes_card_imovel[j][k].img.next.text.strip() )
            valores.append( informacoes_card_imovel[j][k].img['src'][47:-4] )
        
            # Iterando e populando o dicionario
            infos_extras_imovel[cod_imoveis[j]].append(qntd_items)
            infos_extras_imovel[cod_imoveis[j]].append(valores)
        
            qntd_items = []
            valores = []
        
    df_aux = pd.DataFrame.from_dict(infos_extras_imovel, orient='index')
    return df_aux

def data_cleaning(
    card_iterator,
    codigo_imovel,
    local_imovel):
        
    #   print("passei 2.2")
        
    bairro = []
    cidade = []
    
    cod_imoveis = []
    
    for i in range(len(local_imovel)):
        bairro.append(local_imovel[i].split()[0])
        cidade.append(local_imovel[i].split()[1])
        
    for i in range(len(codigo_imovel)):
        cod_imoveis.append(codigo_imovel[i].split()[1])
    
    return bairro, cidade, cod_imoveis
    

In [13]:
# Base librarys
import bs4 as bs4
import urllib.request
import pandas as pd

# AUX librarys
import requests
import time
import re

class Crawler_planeta():
    '''
    Crawler da Imobiliaria planeta, desenvolvido por
    @Carlosal249, o Crawler coleta todas as informacoes disponiveis no endpoint:
    'https://itaivan.com/busca/', e retorna elas em formato .xlsx, podendo ainda ser 
    enviada por E-mail.
    '''
    def __init__(self, qntd_paginas, lista_paginacao):
        self.qntd_paginas = qntd_paginas
        self.lista_paginacao = lista_paginacao
        
        self.headers = {
            "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36", 
            "Accept":"text/html,application/xhtml+xml,application/xml; q=0.9,image/webp,image/apng,*/*;q=0.8"
        }
        
        for i in range(1, qntd_paginas):
            self.lista_paginacao.append( r'https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=' + str(i) )
        
        print(" Objecto Instanciado com Sucesso!")
    
    def crawlear_(self):
        '''
        Funcao especifica para inciar o crawleamento dos dados
        '''
        data_final_01 = pd.DataFrame()
        data_final_02 = pd.DataFrame()
        
        for i in range(0, self.qntd_paginas-1):
            url = self.lista_paginacao[i]
            
            source = urllib.request.urlopen(url).read()
            time.sleep(10)

            soup = bs4.BeautifulSoup(source,'html')
            
            # Instanciando objetos Base da busca
            cards_imovel = soup.find_all('div', class_='slide col-xs-12 col-sm-6 col-md-6 col-lg-4')
            informacoes_card_imovel = [card.find_all('span', class_='tooltip-control') for card in cards_imovel]
    
            teste_imagem = [card.find('div', class_='ls-picture').get('style') for card in cards_imovel]
            imagens_tratadadas = [re.sub('background-image: url', '', imagem) for imagem in teste_imagem]
            tipo_imovel = [card.find('div', class_='ls-categoria').get_text().strip() for card in cards_imovel]
            local_imovel = [card.find('span', class_='ls-loc block').get_text().strip().replace(r' ','') for card in cards_imovel]
            codigo_imovel = [card.find('span', class_='ls-cod').get_text() for card in cards_imovel]
            try:
                m2_imovel = [card.find('span', class_='tooltip-control').get_text().strip() for card in cards_imovel]
                
            except:
                m2_imovel = [0, 0, 0, 0, 0, 0, 0, 0, 0]
                
            try:
                preco_imovel = [card.find('span', class_='block ls-price color').get_text().strip() for card in cards_imovel]
                
            except:
                preco_imovel = [0, 0, 0, 0, 0, 0, 0, 0, 0]
            #print(preco_imovel)
            
            print(f'Passando pela Pagina {self.lista_paginacao[i]}, iteracao N:{i}')
            
            bairro, cidade, cod_imoveis = data_cleaning(
                card_iterator=local_imovel,
                codigo_imovel=codigo_imovel,
                local_imovel=local_imovel
            )
            
            df_aux  = informacoes_auxiliares(
                informacoes_card_imovel = informacoes_card_imovel,
                cod_imoveis =  cod_imoveis
            )
            
            data = pd.DataFrame(list(
                zip(tipo_imovel, bairro, cidade, 
                    cod_imoveis, m2_imovel,
                    preco_imovel,imagens_tratadadas)),
                columns=[
                    'Tipo_imovel', 'bairro', 'cidade',
                    'codigo_imovel', 'm2_imovel',
                    'preco_imovel', 'imagem_imovel'
                ], index=cod_imoveis)
            
            print('')
            
            data_final_01 = pd.concat([data_final_01, data])
            data_final_02 = pd.concat([data_final_02, df_aux])
        
        return data_final_01, data_final_02

In [16]:
lista_pag = []
crawler_head = Crawler_planeta(qntd_paginas=110, lista_paginacao=lista_pag)

df_1, df_2 = crawler_head.crawlear_()

#df1['imagem_imovel'] = df1['imagem_imovel'].str.replace('(','')
#df1['imagem_imovel'] = df1['imagem_imovel'].str.replace(')','')
#df1['imagem_imovel'] = df1['imagem_imovel'].str.replace("'", '')

 Objecto Instanciado com Sucesso!
Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=1, iteracao N:0

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=2, iteracao N:1

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=3, iteracao N:2

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=4, iteracao N:3

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=5, iteracao N:4

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=6, iteracao N:5

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=7, iteracao N:6

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=66, iteracao N:65

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=67, iteracao N:66

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=68, iteracao N:67

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=69, iteracao N:68

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=70, iteracao N:69

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=71, iteracao N:70

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=72, iteracao N:71

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=73,

In [24]:
!pip install XlsxWriter

You should consider upgrading via the 'c:\users\caooc\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [25]:
import xlsxwriter


In [26]:
#df_1_result = df_1.ExcelWriter("BASE_COMPLETA_ITAIVAN_21_07.xlsx", en)
#df_1.to_excel(df_1_result)
#df_1_result.save()


with pd.ExcelWriter('BASE_COMPLETA_ITAIVAN_21_07.xlsx', engine='xlsxwriter') as writer:
    df_1.to_excel(writer, sheet_name='Imoveis')
    df_2.to_excel(writer, sheet_name='Complementos')
    #dataframe3.to_excel(writer, sheet_name='Arts')

In [70]:
for i in range(info_extras_imovel.keys()):
    print(info_extras_imovel[i].values())
    # https://stackoverflow.com/questions/14263872/only-add-to-a-dict-if-a-condition-is-met

TypeError: 'dict_keys' object cannot be interpreted as an integer

In [14]:
df1.to_csv('Integracao_pbi_19_07.csv')
df2.to_csv('Integracao_aux_pbi_19_07.csv')

In [35]:
df1['imagem_imovel'][0]

'https://cdn.vistahost.com.br/itaivan1/vista.imobi/fotos/12363/i3511S3K57mnE04r51wrX_1236362d6e58d41b9a.jpg'

In [9]:
df1.to_csv("INFOS_GERAIS_IMOVEIS_18_07_22.csv")
df2.to_csv("INFOS_DETALHADAS_IMOVEIS_18_07_22.csv")

In [27]:
df1['imagem_imovel'][0]

'https://cdn.vistahost.com.br/itaivan1/vista.imobi/fotos/12363/i3511S3K57mnE04r51wrX_1236362d6e58d41b9a.jpg)'

,0,1,2,3,4,5,6,7,8,9
13213,"[122,70 m²]",[esquadro],[3],[cama],[2],[banheira],[2],[galpao],None,None
13212,"[249,00 m²]",[esquadro],[1],[cama],[1],[cama],[3],[banheira],[2],[galpao]
4207,"[469,00 m²]",[esquadro],None,None,None,None,None,None,None,None
13211,"[170,00 m²]",[esquadro],[2],[cama],[1],[cama],[2],[banheira],[2],[galpao]
13209,"[98,13 m²]",[esquadro],[2],[cama],[1],[cama],[1],[banheira],[1],[galpao]
...,...,...,...,...,...,...,...,...,...,...
13060,"[177,00 m²]",[esquadro],[2],[cama],[1],[cama],[3],[banheira],[2],[galpao]
13059,"[171,90 m²]",[esquadro],[3],[galpao],None,None,None,None,None,None
4067,"[59.925,40 m²]",[esquadro],None,None,None,None,None,None,None,None
74508,"[138,97 m²]",[esquadro],[2],[cama],[1],[cama],[2],[banheira],[1],[galpao]


In [223]:
qntd_items = []
valores = []

infos_extras_imovel = {}

for i in range(0, len(informacoes_card_imovel)):
    infos_extras_imovel.setdefault(cod_imoveis[i], [])
    
    for j in range(0, len(informacoes_card_imovel[i])):
        # Salvando as variaveis em listas
        qntd_items.append( informacoes_card_imovel[i][j].img.next.text.strip() )
        valores.append( informacoes_card_imovel[i][j].img['src'][47:-4] )
        
        # Iterando e populando o dicionario
        infos_extras_imovel[cod_imoveis[i]].append(qntd_items)
        infos_extras_imovel[cod_imoveis[i]].append(valores)
        
        qntd_items = []
        valores = []
        #id_imovel['Codigo_Imovel'] = i
        #id_imovel['qntd_items'].append(qntd_items)
        #id_imovel['valores'].append(valores)

In [7]:


site = 'https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=2'


source = urllib.request.urlopen(site).read()
time.sleep(5)

soup = bs4.BeautifulSoup(source,'html')

cards_imovel = soup.find_all('div', class_='slide col-xs-12 col-sm-6 col-md-6 col-lg-4')
informacoes_card_imovel = [card.find_all('span', class_='tooltip-control') for card in cards_imovel]

In [225]:
df = pd.DataFrame.from_dict(infos_extras_imovel, orient='index')
df_final = pd.DataFrame()
pd.concat([df, df_final])

,0,1,2,3,4,5,6,7,8,9
74646,"[165,00 m²]",[esquadro],[4],[cama],[1],[banheira],[1],[galpao],None,None
4206,"[21.988,70 m²]",[esquadro],None,None,None,None,None,None,None,None
13207,"[24.715,00 m²]",[esquadro],None,None,None,None,None,None,None,None
13206,"[120,00 m²]",[esquadro],[3],[cama],[1],[banheira],[1],[galpao],None,None
74645,"[58,29 m²]",[esquadro],[2],[cama],[1],[banheira],[1],[galpao],None,None
4205,"[153,86 m²]",[esquadro],[2],[cama],[1],[cama],[3],[banheira],[2],[galpao]
13205,"[300,00 m²]",[esquadro],None,None,None,None,None,None,None,None
4198,"[100,00 m²]",[esquadro],[2],[cama],[1],[banheira],[1],[galpao],None,None
4204,"[96,00 m²]",[esquadro],[3],[cama],[1],[banheira],[1],[galpao],None,None


In [8]:
teste_imagem = [card.find('div', class_='ls-picture').get('style') for card in cards_imovel]
tipo_imovel = [card.find('div', class_='ls-categoria').get_text().strip() for card in cards_imovel]
local_imovel = [card.find('span', class_='ls-loc block').get_text().strip().replace(r' ','') for card in cards_imovel]
codigo_imovel = [card.find('span', class_='ls-cod').get_text() for card in cards_imovel]
m2_imovel = [card.find('span', class_='tooltip-control').get_text().strip() for card in cards_imovel]
preco_imovel = [card.find('span', class_='block ls-price color').get_text().strip() for card in cards_imovel]

In [204]:
def data_cleaning(card_iterator=local_imovel,
                  codigo_imovel = codigo_imovel):
    
    bairro = []
    cidade = []
    
    cod_imoveis = []
    
    for i in range(len(local_imovel)):
        bairro.append(local_imovel[i].split()[0])
        cidade.append(local_imovel[i].split()[1])
    
    for i in range(len(codigo_imovel)):
        cod_imoveis.append(codigo_imovel[i].split()[1])
    
    return bairro, cidade, cod_imoveis

In [205]:
bairro, cidade, cod_imoveis = data_cleaning()

In [231]:
import pandas as pd

data = pd.DataFrame(list(
    zip(tipo_imovel, bairro, cidade, 
        cod_imoveis, m2_imovel,
        preco_imovel)),
        columns=[
            'Tipo_imovel', 'bairro', 'cidade',
            'codigo_imovel', 'm2_imovel','preco_imovel']
        , index=cod_imoveis)

data.head()

,Tipo_imovel,bairro,cidade,codigo_imovel,m2_imovel,preco_imovel
74646,Casa,SãoLuís,JaraguáDoSul,74646,"165,00 m²","R$ 437.000,00"
4206,Terreno,VilaBaependi,JaraguáDoSul,4206,"21.988,70 m²","R$ 280.000,00"
13207,Terreno,TifaMartins,JaraguáDoSul,13207,"24.715,00 m²","R$ 280.000,00"
13206,Casa,TrêsRiosDoSul,JaraguáDoSul,13206,"120,00 m²","R$ 712.000,00"
74645,Geminado,Jaraguá99,JaraguáDoSul,74645,"58,29 m²","R$ 300.000,00"


In [232]:
qntd_items = []
valores = []

infos_extras_imovel = {}

for i in range(0, len(informacoes_card_imovel)):
    infos_extras_imovel.setdefault(i, [])
    
    for j in range(0, len(informacoes_card_imovel[i])):
        # Salvando as variaveis em listas
        qntd_items.append( informacoes_card_imovel[i][j].img.next.text.strip() )
        valores.append( informacoes_card_imovel[i][j].img['src'][47:-4] )
        
        # Iterando e populando o dicionario
        infos_extras_imovel[i].append(qntd_items)
        infos_extras_imovel[i].append(valores)
        
        qntd_items = []
        valores = []
        #id_imovel['Codigo_Imovel'] = i
        #id_imovel['qntd_items'].append(qntd_items)
        #id_imovel['valores'].append(valores)

,Tipo_imovel,bairro,cidade,codigo_imovel,m2_imovel,preco_imovel
74646,Casa,SãoLuís,JaraguáDoSul,74646,"165,00 m²","R$ 437.000,00"
4206,Terreno,VilaBaependi,JaraguáDoSul,4206,"21.988,70 m²","R$ 280.000,00"
13207,Terreno,TifaMartins,JaraguáDoSul,13207,"24.715,00 m²","R$ 280.000,00"
13206,Casa,TrêsRiosDoSul,JaraguáDoSul,13206,"120,00 m²","R$ 712.000,00"
74645,Geminado,Jaraguá99,JaraguáDoSul,74645,"58,29 m²","R$ 300.000,00"
4205,Casa,TrêsRiosDoSul,JaraguáDoSul,4205,"153,86 m²","R$ 1.190.000,00"
13205,Terreno,Centro,Schroeder,13205,"300,00 m²","R$ 130.000,00"
4198,Casa,TrêsRiosDoNorte,JaraguáDoSul,4198,"100,00 m²","R$ 426.000,00"
4204,Casa,Itoupava-Açu,Schroeder,4204,"96,00 m²","R$ 586.000,00"


In [84]:
data.to_excel('teste_scraping1.xlsx')